# Pluvial flood hazard

In [ ]:
# Import packages
import subprocess
from pathlib import Path

from hydroflows import Workflow, WorkflowConfig
from hydroflows.log import setuplog
from hydroflows.methods import rainfall, sfincs
from hydroflows.utils.example_data import fetch_data

logger = setuplog(level="INFO")

In [ ]:
# Define case name and root directory
name = "pluvial_hazard"  # for now
pwd = Path().resolve()  # Get the current file location
case_root = Path(pwd, "cases", name)  # output directory


In [ ]:
# Fetch an example sfincs model and store at the output directory
sfincs_root = fetch_data(data="sfincs-model", output_dir=Path(pwd, "cases", name, "models"))

In [ ]:
# Setup the config file
config = WorkflowConfig(
    sfincs_exe=Path(pwd, "bin/sfincs_v2.1.1/sfincs.exe"),
    sfincs_inp=sfincs_root / "sfincs.inp",
    sfincs_region=sfincs_root / "gis" / "region.geojson",
    start_date="2014-01-01",
    end_date="2021-12-31",
    rps=[2, 5, 10],
)

In [ ]:
# Setup the workflow
wf = Workflow(config=config, name=name, root=case_root)

In [ ]:
# Get precipitation data
pluvial_data = rainfall.GetERA5Rainfall(
    region=wf.get_ref("$config.sfincs_region"),
    output_dir="data/era5",
    start_date=wf.get_ref("$config.start_date"),
    end_date=wf.get_ref("$config.end_date"),
)
wf.create_rule(pluvial_data, rule_id="pluvial_data")


In [ ]:
# Derive pluviual events from precipitation data
pluvial_events = rainfall.PluvialDesignEvents(
    precip_nc=pluvial_data.output.precip_nc,
    event_root="data/events",
    rps=wf.get_ref("$config.rps"),
    wildcard="pluvial_events",
)
wf.create_rule(pluvial_events, rule_id="pluvial_events")


In [ ]:
# Update the sfincs model with pluviual events
sfincs_update = sfincs.SfincsUpdateForcing(
    sfincs_inp=wf.get_ref("$config.sfincs_inp"),
    event_yaml=pluvial_events.output.event_yaml,
    output_dir=config.sfincs_inp.parent/"simulations"
)
wf.create_rule(sfincs_update, rule_id="sfincs_update")


In [ ]:
# Run the sfincs model
sfincs_run = sfincs.SfincsRun(
    sfincs_inp=sfincs_update.output.sfincs_out_inp,
    sfincs_exe=wf.get_ref("$config.sfincs_exe"),
)
wf.create_rule(sfincs_run, rule_id="sfincs_run")


In [ ]:
# run workflow
wf.dryrun()


In [ ]:
# Write the workflow to a Snakefile and snakefile.config.yml
wf.to_snakemake()

# show the files in the case directory
print(f"{wf.root.relative_to(pwd)}:")
for f in wf.root.iterdir():
    print(f"- {f.name}")

In [ ]:
from IPython.display import SVG

# (test) run the workflow with snakemake and visualize the directed acyclic graph
# make sure to have snakemake installed in your environment
subprocess.run('snakemake --dag | dot -Tsvg > dag.svg', cwd=wf.root, shell=True).check_returncode()

# show the dag
SVG(Path(wf.root, "dag.svg").as_posix())